In [23]:
from typing import List, Optional

from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import tool
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini")

In [24]:
class Monster(BaseModel):
    """Information about a monster in the Dungeons and Dragons universe."""
    name: Optional[str] = Field(default=None, description="The name of the monster.")
    race: Optional[str] = Field(default=None, description="The race of the monster, for example Orc, Goblin, Dragon, Zombie.")
    features: Optional[str] = Field(default=None, description="What the monster looks like")
    hit_points: Optional[int] = Field(default=None, description="The number of hit points the monster has.")


class Data(BaseModel):
    """Data about a monster in the Dungeons and Dragons universe."""
    monsters: List[Monster]

### Extractor

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are an expert extraction algorighm. "
            "Only extract releant information from the text"
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value",
        ),
        # MessagesPlaceholder("examples"),
        ("human", "{text}"),
    ]
)

In [20]:
runnable = prompt | model.with_structured_output(schema=Monster)

In [21]:
text = """
Goblins were a race of small and numerous goblinoids common throughout Toril, often living in underground caverns near the surface known as lairs. The race was often, though not always, dominated by other goblinoids, most commonly hobgoblins. Goblins may have, in fact, been initially created by this related race to serve as scouts and infiltrators
"""
runnable.invoke({"text": text})

Monster(name='Goblins', race='Goblinoid', features='small and numerous, often living in underground caverns near the surface', hit_points=None)

In [22]:
text = """
Minotaurs were large, muscular humanoids. They had the head of a bull, upper torso, hands, and the body of a human,[20] and hooved feet. Their fur and wild hair typically ranged from brown to black.[4][1] The fur often covered forearms, chest, abdomen, and legs. Males were referred to as bulls and reached up to 9 feet (2.7 meters) in height, while females – cows, were similar in appearance, with broader hips, humanoid breasts, and reached only 7 feet (2.1 meters).[16]

Minotaur bodies were of similar shape and build to that of an ogre. Their hands were huge and strong, ending in thick sharp yellow nails. Minotaurs were known to use claws in battle, but they were brittle and did not survive heavy use. Despite their heads being that of a bull, there were subtle differences in features between mundane beasts and minotaurs. Some were known to have cow-like tails; however, that trait was not common. Their snouts were thicker and longer than that of an animal, and bull-men had sharp carnivore teeth. Their canines were especially noticeable, and minotaurs never hesitated to bite their victims, prey, and opponents. Lastly, unlike bovines, minotaurs' eyes were positioned closer to the center of their faces, granting them good depth perception and another telltale evidence of their predatory nature.[16]

The creatures' iconic feature – their horns – were very similar to normal bulls', slightly curving forward. Both males and females had similarly shaped huge horns. However, males' were longer and thicker on average. There were several horn coloring variations, and sometimes as many as three colors could be present. Minotaurs with dark fur often had dark yellow or brown horns.[16] The shortest horns were around 1 foot (0.3 meters) and could reach up to three times that length
"""
runnable.invoke({"text": text})

Monster(name='Minotaur', race='Humanoid', features='Large, muscular humanoids with the head of a bull, upper torso, hands, and body of a human, hooved feet, fur ranging from brown to black, huge and strong hands with thick sharp yellow nails, thick and long snouts, carnivore teeth, iconic curved horns, males have longer and thicker horns than females.', hit_points=None)

### Multiple entities

In [25]:
runnable = prompt | model.with_structured_output(schema=Data)
text = """
Goblins were a race of small and numerous goblinoids common throughout Toril, often living in underground caverns near the surface known as lairs. The race was often, though not always, dominated by other goblinoids, most commonly hobgoblins. Goblins may have, in fact, been initially created by this related race to serve as scouts and infiltrators

Minotaurs were large, muscular humanoids. They had the head of a bull, upper torso, hands, and the body of a human,[20] and hooved feet. Their fur and wild hair typically ranged from brown to black.[4][1] The fur often covered forearms, chest, abdomen, and legs. Males were referred to as bulls and reached up to 9 feet (2.7 meters) in height, while females – cows, were similar in appearance, with broader hips, humanoid breasts, and reached only 7 feet (2.1 meters).[16]

Minotaur bodies were of similar shape and build to that of an ogre. Their hands were huge and strong, ending in thick sharp yellow nails. Minotaurs were known to use claws in battle, but they were brittle and did not survive heavy use. Despite their heads being that of a bull, there were subtle differences in features between mundane beasts and minotaurs. Some were known to have cow-like tails; however, that trait was not common. Their snouts were thicker and longer than that of an animal, and bull-men had sharp carnivore teeth. Their canines were especially noticeable, and minotaurs never hesitated to bite their victims, prey, and opponents. Lastly, unlike bovines, minotaurs' eyes were positioned closer to the center of their faces, granting them good depth perception and another telltale evidence of their predatory nature.[16]

The creatures' iconic feature – their horns – were very similar to normal bulls', slightly curving forward. Both males and females had similarly shaped huge horns. However, males' were longer and thicker on average. There were several horn coloring variations, and sometimes as many as three colors could be present. Minotaurs with dark fur often had dark yellow or brown horns.[16] The shortest horns were around 1 foot (0.3 meters) and could reach up to three times that length
"""

runnable.invoke({"text": text})

Data(monsters=[Monster(name='Goblin', race='Goblinoid', features='Small and numerous, often living in underground caverns, dominated by hobgoblins', hit_points=None), Monster(name='Minotaur', race='Humanoid', features='Large, muscular humanoids with the head of a bull, upper torso and body of a human, hooved feet, and fur ranging from brown to black', hit_points=None)])

### Examples

https://python.langchain.com/v0.2/docs/how_to/extraction_examples/

In [26]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are an expert extraction algorighm. "
            "Only extract releant information from the text"
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value",
        ),
        MessagesPlaceholder("examples"),
        ("human", "{text}"),
    ]
)

In [27]:
prompt.invoke(
    {
        "text": "Goblins were a race of small and numerous goblinoids common throughout Toril, often living in underground caverns near the surface known as lairs. The race was often, though not always, dominated by other goblinoids, most commonly hobgoblins. Goblins may have, in fact, been initially created by this related race to serve as scouts and infiltrators",
        "examples": [
            HumanMessage(content="testing 1 2 3"),
        ],
    }
)

ChatPromptValue(messages=[SystemMessage(content="You are an expert extraction algorighm. Only extract releant information from the textIf you do not know the value of an attribute asked to extract, return null for the attribute's value"), HumanMessage(content='testing 1 2 3'), HumanMessage(content='Goblins were a race of small and numerous goblinoids common throughout Toril, often living in underground caverns near the surface known as lairs. The race was often, though not always, dominated by other goblinoids, most commonly hobgoblins. Goblins may have, in fact, been initially created by this related race to serve as scouts and infiltrators')])

In [30]:
examples = [
    (
        "Zombies are undead creatures of the night. They are slow, cant speak and humaniod. Skeletons are also undead creatures, but they have no flesh. Both are spawend from the underworld",
        Data(monsters=[
            Monster(name="Zombie", race="Undead", features="humanoid, slow and can't speak"),
            Monster(name="Skeleton", race="Undead", features="no flesh"),
        ]),
    ),
]

messages = list()
